In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/test.csv')
test.head()

In [ ]:
submission = pd.read_csv('/kaggle/input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
submission.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.language.unique()

In [ ]:
train.language.value_counts()

In [ ]:
import string
import re
string.punctuation
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train.context = train.context.apply(lambda x: clean_text(x))

In [ ]:
train.context

In [ ]:
test.context = test.context.apply(lambda x: clean_text(x))
test.context

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/xlm-roberta-large-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

QA_input = {
    'question': train['question'][1],
    'context': train['context'][1]
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
answer = []
for i in range(len(test)):
    QA_input = {
    'question': test['question'][i],
    'context': test['context'][i]
    }
    answer.append(nlp(QA_input)['answer'])

In [ ]:
test['PredictionString'] = answer
del test['context']
del test['question']
del test['language']

test.to_csv('submission1.csv', index=False)